# Forecast to Power Tutorial

This tutorial will walk through the process of going from a WRF forecast model data to AC power using the SAPM.

Table of contents:
1. [Setup](#Setup)
2. [Load Forecast data](#Load-Forecast-data)
2. [Calculate modeling intermediates](#Calculate-modeling-intermediates)
2. [DC power using SAPM](#DC-power-using-SAPM)
2. [AC power using SAPM](#AC-power-using-SAPM)

This tutorial requires pvlib >= 0.6.0.

Authors:
* Jeffrey Sward (jas983@cornell.edu), Cornell University, June 2020

## Setup

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

# built in python modules
import datetime
import os
import inspect
import sys

# python add-ons
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import wrf
from optwrf import runwrf as rw

# # Import the pvlib module
if sys.platform == 'linux':
    sys.path.append('/home/jsward/Documents/01_Research/01_Renewable_Analysis/WRF/pvlib-python')
import pvlib
from pvlib.wrfcast import WRF

/Users/swardy9230/Box Sync/01_Research/01_Renewable_Analysis/WRF_Solar_and_Wind/pvlib-python/pvlib/wrfcast.py:20: UserWarning: The forecast module algorithms and features are highly experimental. The API may change, the functionality may be consolidated into an io module, or the module may be separated into its own package.
  'The forecast module algorithms and features are highly experimental. '


## Load Forecast Data

In [2]:
# Specify the forecast model as WRF
fm = WRF()

In [4]:
# Find the absolute file path to your pvlib installation
pvlib_abspath = os.path.dirname(os.path.abspath(inspect.getfile(pvlib)))

# absolute path to WRF data file
DIR_WRFOUT = os.path.join(pvlib_abspath, 'data/')
wrfout_file = 'wrfout_d01_2011-01-24_01:00:00'

# Get/process the data from the wrfout file
forecast_data = rw.process_wrfout_data(DIR_WRFOUT, wrfout_file, save_file=False)

Below is an output of the processed WRF data

In [6]:
forecast_data.head()

<xarray.Dataset>
Dimensions:         (Time: 5, bottom_top: 5, south_north: 5, west_east: 5)
Coordinates:
  * Time            (Time) datetime64[ns] 2011-01-24T01:00:00 ... 2011-01-24T05:00:00
    lon             (south_north, west_east) float32 -95.43335 ... -94.920715
    lat             (south_north, west_east) float32 30.952759 ... 31.37416
    wspd_wdir       <U4 'wspd'
    XLONG           (south_north, west_east) float32 -95.43335 ... -94.920715
    XLAT            (south_north, west_east) float32 30.952759 ... 31.37416
    XTIME           (Time) float32 780.0 840.0 900.0 960.0 1020.0
    level           int64 100
Dimensions without coordinates: bottom_top, south_north, west_east
Data variables:
    Times           (Time) datetime64[ns] 2011-01-24T01:00:00 ... 2011-01-24T05:00:00
    temp_air        (Time, south_north, west_east) float32 14.474274 ... 11.201599
    wind_speed_u    (Time, south_north, west_east) float32 0.31499797 ... 2.2448149
    wind_speed_v    (Time, south_north, west_east) float32 1.1208413 ... -0.050445393
    cloud_fraction  (Time, bottom_top, south_north, west_east) float32 0.0 ... 0.0
    cos_zenith      (Time, south_north, west_east) float32 -0.23945779 ... -0.9047607
    dni             (Time, south_north, west_east) float32 0.0 0.0 ... 0.0 0.0
    dhi             (Time, south_north, west_east) float32 0.0 0.0 ... 0.0 0.0
    ghi             (Time, south_north, west_east) float32 0.0 0.0 ... 0.0 0.0
    wind_speed10    (Time, south_north, west_east) float32 1.1642632 ... 2.2453816
    wind_speed100   (Time, south_north, west_east) float32 2.304195 ... 4.2133384
    wpd             (Time, south_north, west_east) float32 6116.8477 ... 37398.055

This is an ``xarray DataSet`` object. It has a lot of great properties that are beyond the scope of our tutorials. Specifically for those working with 3-dimensional (or higher) data.